In [4]:
import deepxde as dde
import numpy as np
import tensorflow as tf

# Define the PDE
def pde(x, u):
    du_t = dde.grad.jacobian(u, x, i=0, j=1)
    du_xx = dde.grad.hessian(u, x, i=0, j=0)
    return du_t - du_xx

# Geometry and time domain
geom = dde.geometry.Interval(0, 1)
timedomain = dde.geometry.TimeDomain(0, 1)
geomtime = dde.geometry.GeometryXTime(geom, timedomain)

# Initial condition
def func_ic(x):
    return np.sin(np.pi * x[:, 0:1])

ic = dde.IC(geomtime, func_ic, lambda _, on_initial: on_initial)

# Boundary condition
bc = dde.DirichletBC(geomtime, dde.icbc.func_bcs, lambda _, on_boundary: on_boundary)

# Define the data object
data = dde.data.TimePDE(geomtime, pde, [ic, bc], num_domain=2540, num_boundary=80, num_initial=160)

# Define the neural network
layer_size = [2] + [32] * 3 + [1]
activation = "tanh"
initializer = "Glorot uniform"
net = dde.maps.FNN(layer_size, activation, initializer)

# Define the model
model = dde.Model(data, net)

# Compile and train the model
model.compile("adam", lr=0.001)
losshistory, train_state = model.train(epochs=5000)

# Plot the results
dde.saveplot(losshistory, train_state, issave=True, isplot=True)


RuntimeError: Backend is set as tensorflow.compat.v1, but 'tensorflow.compat.v1' failed to import.